In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications import DenseNet121
import datetime

class MyCustomCallback(tf.keras.callbacks.Callback):

  def on_train_batch_begin(self, batch, logs=None):
    print('Training: batch {} begins in {}'.format(batch, datetime.datetime.now().time()))

  def on_train_batch_end(self, batch, logs=None):
    print('Training: batch {} ends in {}'.format(batch, datetime.datetime.now().time()))

  def on_test_batch_begin(self, batch, logs=None):
    print('Validation: batch {} begins in {}'.format(batch, datetime.datetime.now().time()))

  def on_test_batch_end(self, batch, logs=None):
    print('Validation: batch {} ends in {}'.format(batch, datetime.datetime.now().time()))


directory = "../input/editeddatancml/seg_train_small/seg_train/" #(Alex) Fixed it! now it should work


seed_value=123
train = tf.keras.preprocessing.image_dataset_from_directory(
    directory, labels='inferred', label_mode='categorical', seed=seed_value, color_mode='rgb', batch_size=32, image_size=(150,150), shuffle=True, validation_split=0.25, 
    subset="training")

val = tf.keras.preprocessing.image_dataset_from_directory(
    directory, labels='inferred', label_mode='categorical', seed=seed_value,
    color_mode='rgb', batch_size=32, image_size=(150,150), shuffle=True, validation_split=0.25, subset="validation")

##A: Here it throws an error, seems to need more args ( return DenseNet([6, 12, 24, 16], include_top, weights, input_tensor,
# input_shape, pooling, classes))
base_model = DenseNet121(include_top=False, weights="imagenet", input_shape=(150,150,3)) 
inputs = keras.Input(shape=(150, 150, 3))


#prediction = Dense(6, activation='soft_max')(x)
base_model.trainable= False

x = inputs
norm_layer = keras.layers.experimental.preprocessing.Normalization()
mean = np.array([127.5] * 3)
var = mean ** 2
# Scale inputs to [-1, +1]
x = norm_layer(x)
norm_layer.set_weights([mean, var])

#Maybe add an extra dense layer before
x = base_model(x, training=False)
#x = keras.layers.GlobalAveragePooling2D()(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(1024, activation='relu')(x)
x = keras.layers.Dropout(0.35)(x) #should i have dropout layer?
x = keras.layers.Dense(1024, activation='relu')(x)
outputs = keras.layers.Dense(6)(x)

model = keras.Model(inputs, outputs)
model.summary()

model.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.CategoricalCrossentropy(from_logits=True),
    metrics=[keras.metrics.CategoricalAccuracy()],
)

epochs = 20
model.fit(train, epochs=epochs, validation_data=val)


_ = model.fit(x_train, y_train,
          batch_size=32,
          epochs=1,
          steps_per_epoch=5,
          verbose=0,
          callbacks=[MyCustomCallback()])